In [1]:
import sys; sys.path.append('..')
import matplotlib.pyplot as plt
import quimb.tensor as qtn
import cotengra as ctg
import quimb as qu
import numpy as np
import time, random

In [4]:
import re
import math
import numbers
import operator
import functools
import itertools

import numpy as np
from autoray import do, reshape

import quimb as qu
from quimb.utils import oset, partitionby, concatv, partition_all, ensure_dict, LRU

In [5]:
qasm_str = """4
0 hz_1_2 0 
0 x_1_2 1 
0 x_1_2 2 
0 hz_1_2 3 
1 rz 0 1.210696502097985
1 rz 1 2.4326562950300605
1 rz 2 -2.225882728378087
1 rz 3 -2.7293249642087485
2 fs 0 2 1.23123 1.23123
2 fs 1 3 1.53079632679 0.47689957001758815
3 rz 1 -2.3932854391951004
3 rz 2 2.600059005847074
3 rz 3 2.1293419565556686
3 rz 0 2.635214888513154
4 y_1_2 0 
4 y_1_2 1 
4 y_1_2 2 
4 x_1_2 3
"""

In [6]:
def _convert_ints_and_floats(x):
    if isinstance(x, str):
        try:
            return int(x)
        except ValueError:
            pass

        try:
            return float(x)
        except ValueError:
            pass

    return x


def _put_registers_last(x):
    # no need to do anything unless parameter (i.e. float) is found last
    if not isinstance(x[-1], float):
        return x

    # swap this last group of floats with the penultimate group of integers
    parts = tuple(partitionby(type, x))
    return tuple(concatv(*parts[:-2], parts[-1], parts[-2]))

In [7]:
def parse_qasm(qasm):
    """Parse qasm from a string.

    Parameters
    ----------
    qasm : str
        The full string of the qasm file.

    Returns
    -------
    circuit_info : dict
        Information about the circuit:

        - circuit_info['n']: the number of qubits
        - circuit_info['n_gates']: the number of gates in total
        - circuit_info['gates']: list[list[str]], list of gates, each of which
          is a list of strings read from a line of the qasm file.
    """

    lines = qasm.split('\n')
    n = int(lines[0])

    # turn into tuples of python types
    gates = [
        tuple(map(_convert_ints_and_floats, line.strip().split(" ")))
        for line in lines[1:] if line
    ]

    # put registers/parameters in standard order and detect if gate round used
    gates = tuple(map(_put_registers_last, gates))
    round_specified = isinstance(gates[0][0], numbers.Integral)

    return {
        'n': n,
        'gates': gates,
        'n_gates': len(gates),
        'round_specified': round_specified,
    }

In [8]:
parse_qasm(qasm_str)

{'n': 4,
 'gates': ((0, 'hz_1_2', 0),
  (0, 'x_1_2', 1),
  (0, 'x_1_2', 2),
  (0, 'hz_1_2', 3),
  (1, 'rz', 1.210696502097985, 0),
  (1, 'rz', 2.4326562950300605, 1),
  (1, 'rz', -2.225882728378087, 2),
  (1, 'rz', -2.7293249642087485, 3),
  (2, 'fs', 1.23123, 1.23123, 0, 2),
  (2, 'fs', 1.53079632679, 0.47689957001758815, 1, 3),
  (3, 'rz', -2.3932854391951004, 1),
  (3, 'rz', 2.600059005847074, 2),
  (3, 'rz', 2.1293419565556686, 3),
  (3, 'rz', 2.635214888513154, 0),
  (4, 'y_1_2', 0),
  (4, 'y_1_2', 1),
  (4, 'y_1_2', 2),
  (4, 'x_1_2', 3)),
 'n_gates': 18,
 'round_specified': True}

In [ ]:
openqasm_str = """OPENQASM 2.0;
include "qelib1.inc";
qreg qubits[5];
h qubits[4];
h qubits[4];
ccx qubits[2],qubits[3],qubits[4];
h qubits[4];
h qubits[4];
h qubits[3];
h qubits[3];
ccx qubits[0],qubits[1],qubits[3];
h qubits[3];
h qubits[3];
h qubits[4];
h qubits[4];
ccx qubits[2],qubits[3],qubits[4];
h qubits[4];
h qubits[4];
h qubits[3];
h qubits[3];
ccx qubits[0],qubits[1],qubits[3];
h qubits[3];
h qubits[3];
"""